<a href="https://colab.research.google.com/github/tyukei/Aozora-transformer/blob/main/Aozora_llm_uv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!curl -LsSf https://astral.sh/uv/install.sh | sh

downloading uv 0.8.13 x86_64-unknown-linux-gnu
no checksums to verify
installing to /usr/local/bin
  uv
  uvx
everything's installed!


In [2]:
!uv --version

uv 0.8.13


In [3]:
!git clone https://github.com/speed1313/jax-llm.git
%cd jax-llm

Cloning into 'jax-llm'...
remote: Enumerating objects: 409, done.
remote: Counting objects: 100% (60/60), done.
remote: Compressing objects: 100% (33/33), done.
remote: Total 409 (delta 24), reused 45 (delta 20), pack-reused 349 (from 1)
Receiving objects: 100% (409/409), 558.25 KiB | 4.77 MiB/s, done.
Resolving deltas: 100% (229/229), done.
/content/jax-llm


In [4]:
!uv lock --prerelease=allow
!uv sync --frozen

Streaming output truncated to the last 5000 lines.
nvidia-cusolver-cu12               ------------------------------ 55.89 MiB/255.11 MiB
nvidia-cusparselt-cu12             ------------------------------ 56.26 MiB/273.89 MiB
nvidia-cusparse-cu12               ------------------------------ 55.57 MiB/274.86 MiB
nvidia-nccl-cu12                   ------------------------------ 55.66 MiB/307.43 MiB
⠼ Preparing packages... (116/129)
pyarrow                            ------------------------------ 40.04 MiB/40.84 MiB
nvidia-curand-cu12                 ------------------------------ 55.74 MiB/60.67 MiB
jaxlib                             ------------------------------ 55.93 MiB/77.39 MiB
nvidia-cuda-nvrtc-cu12             ------------------------------ 55.44 MiB/83.96 MiB
triton                             ------------------------------ 56.32 MiB/148.33 MiB
nvidia-cufft-cu12                  ------------------------------ 56.08 MiB/184.17 MiB
nvidia-cusolver-cu12               --------------

In [5]:
!uv pip install --prerelease=allow "jax[tpu]"

Using Python 3.12.11 environment at: /usr
Audited 1 package in 1.05s


In [6]:
import jax, jaxlib
print("jax    :", jax.__version__)
print("jaxlib :", jaxlib.__version__)
print("devices:", [d.platform for d in jax.devices()])

jax    : 0.5.3
jaxlib : 0.5.3
devices: ['tpu', 'tpu', 'tpu', 'tpu', 'tpu', 'tpu', 'tpu', 'tpu']


In [7]:
!python /content/jax-llm/src/jax_llm/prepare_aozora.py --book_num 10

Traceback (most recent call last):
  File "/content/jax-llm/src/jax_llm/prepare_aozora.py", line 1, in <module>
    from datasets import load_dataset
ModuleNotFoundError: No module named 'datasets'


In [8]:
# %env JAX_PLATFORMS=cpu
!python /content/jax-llm/src/jax_llm/train_tokenizer.py --data_name "aozora_10"

Traceback (most recent call last):
  File "/content/jax-llm/src/jax_llm/train_tokenizer.py", line 44, in <module>
    main()
  File "/usr/local/lib/python3.12/dist-packages/click/core.py", line 1442, in __call__
    return self.main(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/click/core.py", line 1363, in main
    rv = self.invoke(ctx)
         ^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/click/core.py", line 1226, in invoke
    return ctx.invoke(self.callback, **ctx.params)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/click/core.py", line 794, in invoke
    return callback(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/content/jax-llm/src/jax_llm/train_tokenizer.py", line 23, in main
    tokenizer.train(files, trainer)
Exception: No such file or directory (os error 2)


In [ ]:
# %env JAX_PLATFORMS=tpu
# !pip uninstall -y jax jaxlib libtpu
# !pip install -U "jax[tpu]==0.4.33" -f https://storage.googleapis.com/jax-releases/libtpu_releases.html
!python /content/jax-llm/src/jax_llm/train.py --data_name "aozora_10246" --batch_size 32 --n_iterations 5000 --n_freq_eval 100 --dropout_rate 0.1 --learning_rate 0.001 --num_layers 12 --embed_size 512  --head_size 64 --num_heads 8 --block_size 256

In [ ]:
!python /content/jax-llm/src/jax_llm/generate.py --prompt "あなたは誰？" --data_name "aozora_500" --max_new_tokens 200

In [ ]:
!python /content/jax-llm/src/jax_llm/generate.py --prompt "吾輩は猫で" --data_name "aozora_500" --max_new_tokens 200

In [ ]:
!python /content/jax-llm/src/jax_llm/generate.py --prompt "昔々あるところに、" --data_name "aozora_500" --max_new_tokens 200